<a href="https://colab.research.google.com/github/RazerRaymond/Pricing-Simulations/blob/main/conditional_MC_forward_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
import scipy.stats as st

Conditional Monte Carlo for a forward start option

In [ ]:
# Data for Stock
S0 = 120  # spot price
r = 0.025  # risk-free interest rate
q = 0.0175 # div rate
sigma = 0.35  # volatility of stock price
T = 1.0  # maturity date

mu = r - q

In [ ]:
# Option data
H1 = 100.0
H2 = 150.0
lamb1 = 0.9
lamb2 = 1.1
rebate = 10.0

In [ ]:
# CI data
alpha = 0.01
z = st.norm.ppf(1-alpha/2)

In [ ]:
def BMS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black Merton Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * 
np.sqrt(tau))
    return d1
def BMS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black Merton Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * 
np.sqrt(tau))
    return d2
def BMS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black Merton Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    d2 = BMS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price

In [ ]:
def gbm(S0, mu, sigma, dt):
     return S0 * np.exp((mu - (sigma**2) / 2) * dt + sigma * np.sqrt(dt) * np.random.normal())

def h_opt(S0, H1, H2, lamb1, lamb2, rebate):
    ST4 = gbm(S0, mu, sigma, T/4)
    if ST4 < H1:
      K = lamb1 * ST4
      ST = gbm(ST4, mu, sigma, T - T/4)
      return np.maximum(0, (K - ST)) * np.exp(-r * T)
    elif ST4 > H2:
      K = lamb2 * ST4
      ST = gbm(ST4, mu, sigma, T - T/4)
      return np.maximum(0, (ST - K)) * np.exp(-r * T)
    else:
      return rebate * np.exp(-r * T/4)

def h_opt_cmc(S0, H1, H2, lamb1, lamb2, rebate):
    ST4 = gbm(S0, mu, sigma, T/4)
    if ST4 < H1:
      K = lamb1 * ST4
      return BMS_price('put', ST4, K, r, q, sigma, T, T/4) * np.exp(-r * T/4)
    elif ST4 > H2:
      K = lamb2 * ST4
      return BMS_price('call', ST4, K, r, q, sigma, T, T/4) * np.exp(-r * T/4)
    else:
      return rebate * np.exp(-r * T/4)

In [ ]:
# Standard Simulation
simulation_count = 50000
standard_MC = np.zeros(simulation_count)
CMC = np.zeros(simulation_count)
for i in range(simulation_count):
  standard_MC[i] = h_opt(S0, H1, H2, lamb1, lamb2, rebate)
  CMC[i] = h_opt_cmc(S0, H1, H2, lamb1, lamb2, rebate)

In [ ]:
theta_bar = np.mean(standard_MC)
sigma_bar = np.std(standard_MC, ddof = 1)
CI_l = theta_bar - (z * sigma_bar / np.sqrt(simulation_count))
CI_h = theta_bar + (z * sigma_bar / np.sqrt(simulation_count))
print(f"Estimated Option price for h using Standard Simulation: {theta_bar}")
print(f"Estimated Option standard deviation for h using Standard Simulation: {sigma_bar}")
print(f"99 percent CI for Standard Simulation: [{CI_l}, {CI_h}]")

Estimated Option price for h using Standard Simulation: 9.617281323729964
Estimated Option standard deviation for h using Standard Simulation: 9.492469005887893
99 percent CI for Standard Simulation: [9.507933217704426, 9.726629429755501]


In [ ]:
theta_bar = np.mean(CMC)
sigma_bar = np.std(CMC, ddof = 1)
CI_l = theta_bar - (z * sigma_bar / np.sqrt(simulation_count))
CI_h = theta_bar + (z * sigma_bar / np.sqrt(simulation_count))
print(f"Estimated Option price for h using Conditional Monte Carlo Simulation: {theta_bar}")
print(f"Estimated Option standard deviation for h using Monte Carlo Simulation: {sigma_bar}")
print(f"99 percent CI for Standard Simulation: [{CI_l}, {CI_h}]")

Estimated Option price for h using Conditional Monte Carlo Simulation: 9.627756843914323
Estimated Option standard deviation for h using Monte Carlo Simulation: 1.8757428754398984
99 percent CI for Standard Simulation: [9.606149301061876, 9.64936438676677]


As we can tell, using the Conditional Monte Carlo trick, our standard deviation is much smaller, resulting a smaller Confidence Interval.